In [21]:

import torchvision 
import torch 
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [9]:
!pip install kaggle

In [10]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"rmham93","key":"66868e22268433e2f5c1220f2504477a"}'}

In [11]:
!mkdir -p ~/ .kaggle
!cp -r kaggle.json ~/ .kaggle/
!chmod 600 ~/ .kaggle/kaggle.json

In [12]:
!mv .kaggle /root/

mv: cannot move '.kaggle' to '/root/.kaggle': Directory not empty


In [13]:
!kaggle datasets download -d rahulbhalley/fashion-usage

fashion-usage.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!ls

fashion-usage.zip  kaggle.json	sample_data


In [14]:
!unzip fashion-usage.zip

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: usage/Formal/45308.jpg  
  inflating: usage/Formal/45309.jpg  
  inflating: usage/Formal/45310.jpg  
  inflating: usage/Formal/45312.jpg  
  inflating: usage/Formal/45313.jpg  
  inflating: usage/Formal/45315.jpg  
  inflating: usage/Formal/45317.jpg  
  inflating: usage/Formal/45318.jpg  
  inflating: usage/Formal/45574.jpg  
  inflating: usage/Formal/45575.jpg  
  inflating: usage/Formal/45576.jpg  
  inflating: usage/Formal/45584.jpg  
  inflating: usage/Formal/45585.jpg  
  inflating: usage/Formal/45586.jpg  
  inflating: usage/Formal/45587.jpg  
  inflating: usage/Formal/45588.jpg  
  inflating: usage/Formal/45589.jpg  
  inflating: usage/Formal/45590.jpg  
  inflating: usage/Formal/45591.jpg  
  inflating: usage/Formal/45592.jpg  
  inflating: usage/Formal/45593.jpg  
  inflating: usage/Formal/45595.jpg  
  inflating: usage/Formal/45596.jpg  
  inflating: usage/Formal/45597.jpg  
  inflating: usage/Formal/45598.jpg  
  i

In [8]:
!ls

fashion-usage.zip  kaggle.json	sample_data  usage


In [20]:
!cd /content/usage/
!pwd

/content


In [25]:
!rm -r /content/usage/NA

rm: cannot remove '/content/usage/NA': No such file or directory


In [26]:
!rm -r /content/usage/Home

In [27]:
model = models.resnet50(pretrained=True) #resneti indirelim

for isim, param in model.named_parameters():
    
    if 'layer4' not in isim:
      param.requires_grad = False

model.fc = torch.nn.Linear(2048, 7)

In [28]:
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])

In [34]:
dataset = datasets.ImageFolder('/root/content/usage', transform=train_transforms)

dataset_size= len(dataset)

train_size = int(dataset_size * 0.75)

test_boyutu = dataset_size - train_size

train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_boyutu])

print('Dataset boyutu: ', len(dataset))

FileNotFoundError: ignored